In [1]:
# power = 260mW 
# T = 64 C 
%matplotlib ipympl 
from pianoq.lab.asi_cam import ASICam
from pianoq.lab.dmd import DMD
import numpy as np 

In [37]:
cam = ASICam()

In [34]:
cam.close()

In [56]:
path = r'G:\My Drive\Projects\DMD\quantum\Results\Callibrations'
# cam.set_roi(4300, 2100, 500, 500)
cam.set_exposure(3)
cam.set_roi(None, None, None, None)
cam.set_binning(4)
# cam.show_image(vmin=None, vmax=None)
cam.save_image(rf'{path}\farfield_plane_yes_lens_75mm_f1_200mm.fits')

In [2]:
dmd = DMD()

Connected!
Built project!


In [13]:
dmd.set_white()

In [8]:
dmd.set_black()

In [12]:
dmd.set_grating(500)